# 파이토치로 Linear Layer 구현

## Pytorch 모델 용어 정리

* 계층(layer) : 모듈 또는 모듈을 구성하는 한 개의 계층을 의미함 (예: 선형 계층, Linear Layer)
* 모듈(module) : 한 개 이상의 계층이 모여 구성된 것. 모듈이 모여서 새로운 모듈 구성 가능
* 모델(model) : 최종적인 네트워크, 한 개의 모듈이 모델이 될 수도 있고, 여러 개의 모듈이 하나의 모델이 될 수 있음

## torch.nn 과 nn.Module

* torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소 제공
* 모든 PyTorch 모듈은 nn.Module의 하위 클래스(subclass) 임
* 모든 PyTorch 신경망 모델은 nn.Module을 상속 받은 하위 클래스로 정의함

  * `__init__` 에서 신경망 계층 초기화 필요
  * forward() 메서드에서 입력 데이터에 대한 연산 정의 필요

## Raw-level Linear Layer 구현

* 벡터 X 행렬은 벡터의 차원 수를 (a), 행렬의 열의 수 (b, c)일 때, a와 b가 같아야하고,
* 이때 결과 shape 는 (a) x (a, c) = (c)가됨
* (4) x (4, 3) = (3)이 되고, (3) + (3) = (3)이 될 것임

In [1]:
import torch

In [2]:
x = torch.FloatTensor(4) # 입력
W = torch.FloatTensor(4, 3) # 가중치
b = torch.FloatTensor(3) # 편향

In [3]:
# Linear Layer 함수를 정의한다면
def linearFunction(x, W, b):
  y = torch.matmul(x, W) + b
  return y

In [6]:
print("W" , W.shape, W)
print("b" , b.shape, b)
y = linearFunction(x, W, b)
print("\n")
print("y" , y.shape, y)

W torch.Size([4, 3]) tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
b torch.Size([3]) tensor([0., 0., 0.])


y torch.Size([3]) tensor([0., 0., 0.])


### nn.Module 기반, Linear Layer 구현

* 신경망 모델 클래스를 만들고, nn.Module을 상속받음
* `__init__`에서 신경망 계층 초기화 선언
* forward() 메서드에서 입력 데이터에 대한 연산 정의

In [8]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
  def __init__(self):
    # super() 함수는 super(subclass 이름, subclass 객체).__init__() 와 같이 써야 하지만,
    # 하부 클래스 선언 내부에서 super() 호출 시는 super().__init__()와 같이 쓰면, 자동으로 두 인자가 넣어져서 호출됨
    super().__init__()
    # __init__() 에서 신경망 계층 초기화
    self.W = torch.FloatTensor(4, 3)
    self.b = torch.FloatTensor(3)

  def forward(self, x):
    # |x| = (input_dim)
    # |y| = (input_dim) * (input_dim, output_dim) + (output_dim) = (output_dim)
    y = torch.matmul(x, self.W) + self.b
    return y

In [14]:
x = torch.FloatTensor(4)
myLinear = NeuralNetwork()
# forward에 넣을 인자값으로 호출하면, 내부적으로 forward() 함수가 자동 호출함 (정석적 방법)
# 내부 처리중, forward() 전처리/ 후처리도 수행하므로, forward()를 직접 호출하면, 전처리/후처리를 수행하지 않게 될 수 있음
y = myLinear(x)
print(y, y.shape)

tensor([7.5095e-09, 1.5638e-42, 2.3694e-38]) torch.Size([3])


In [16]:
# 개선된 Linear
class NeuralNetwork(nn.Module):
  def __init__(self, input_dim, output_dim):
    # super() 함수는 super(subclass 이름, subclass 객체).__init__() 와 같이 써야 하지만,
    # 하부 클래스 선언 내부에서 super() 호출 시는 super().__init__()와 같이 쓰면, 자동으로 두 인자가 넣어져서 호출됨
    super().__init__()
    # __init__() 에서 신경망 계층 초기화
    self.W = torch.FloatTensor(input_dim, output_dim)
    self.b = torch.FloatTensor(output_dim)

  def forward(self, x):
    # |x| = (input_dim)
    # |y| = (input_dim) * (input_dim, output_dim) + (output_dim) = (output_dim)
    y = torch.matmul(x, self.W) + self.b
    return y

In [19]:
x = torch.FloatTensor(4)
myLinear = NeuralNetwork(4, 3)
y = myLinear(x)
print(y, y.shape)

tensor([0.0000, 0.1876, 0.0000]) torch.Size([3])


In [21]:
for param in myLinear.parameters():
  print(param)

### nn.Module 기반, nn.Parameter 등록하기

* 학습 대상이 되는 텐서는 해당 모듈에 연결된 Parameter로 등록해야 함

  * 이를 통해 특정 모듈에서 학습 처리시 필요한 작업을 알아서 해주도록 구성되어 있음
  * 모듈 내에서 학습 대상이 되는 텐서들은 `__init__`에서 nn.Parameter()으로 등록해줘야 함

    * nn.Parameter(텐서, requires_grad = True)

      * requires_grad 는 디폴트는 True이며, 파라미터가 gradient(점진적으로 최적값을 찾아가는 방식) 방식으로 계산되는 케이스를 의미함
      * 내부적으로 gradient 방식에서 필요한 미분 연산을 위해, 파이토치는 동적으로 그래프를 구성하므로, 이런 구성도 필요한 파라미터임을 의미함

* 기본 신경망 모델 코드 작성 방법

  1. 신경망 모델 클래스를 만들고, nn.Module을 상속받음
  2. `__init__`에서 신경망 계층 초기화 선언 (모듈 내에서 학습 대상이 되는 텐서들은 nn.Parameter()으로 등록)
  3. forward() 메서드에서 입력 데이터에 대한 연산 정의

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self, input_dim, output_dim):
    # super() 함수는 super(subclass 이름, subclass 객체).__init__() 와 같이 써야 하지만,
    # 하부 클래스 선언 내부에서 super() 호출 시는 super().__init__()와 같이 쓰면, 자동으로 두 인자가 넣어져서 호출됨
    super().__init__()
    # __init__() 에서 신경망 계층 초기화
    self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
    self.b = nn.Parameter(torch.FloatTensor(output_dim))

  def forward(self, x):
    # |x| = (input_dim)
    # |y| = (input_dim) * (input_dim, output_dim) + (output_dim) = (output_dim)
    y = torch.matmul(x, self.W) + self.b
    return y